# bge-m3嵌入

In [1]:
%%time

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="bge-m3",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], 
    # show_progress=True
)
len(pass_embedding[1])

CPU times: user 1.4 s, sys: 253 ms, total: 1.65 s
Wall time: 3.37 s


1024

In [2]:
%%time
%%capture

!pip install llama-index
!pip install llama-index-llms-openai-like
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-faiss

CPU times: user 32.2 ms, sys: 16.8 ms, total: 49 ms
Wall time: 7.45 s


In [3]:
%%time

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

CPU times: user 2.65 ms, sys: 455 µs, total: 3.1 ms
Wall time: 2.62 ms


In [4]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 275 µs, sys: 0 ns, total: 275 µs
Wall time: 278 µs


In [5]:
%%time

Settings.embed_model = ollama_embedding

CPU times: user 617 ms, sys: 43.8 ms, total: 661 ms
Wall time: 661 ms


In [6]:
%%time

documents = SimpleDirectoryReader("./books/").load_data()
documents

CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 9.58 ms


[Document(id_='1d0663c0-f2e3-4330-bc42-1088db6a92c8', embedding=None, metadata={'file_path': '/root/notebook/my-jupyter-notebook/llm/bge/books/iphone.txt', 'file_name': 'iphone.txt', 'file_type': 'text/plain', 'file_size': 563, 'creation_date': '2024-05-07', 'last_modified_date': '2024-05-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='iPhone 15和iPhone 15 Plus是由Apple Inc.设计、开发和销售的智能手机。它们是第十七代、也是当前一代iPhone，接替iPhone 14 和 iPhone 14 Plus。这些设备于 2023 年 9 月 12 日在加利福尼亚州库比蒂诺Apple Park举行的Apple 活动期间与价格较高的iPhone 15 Pro 和 15 Pro Max一起发布。预订于 2023 年 9 月 15 日开始，这些设备于 2023 年 9 月 22 日上市。与 iPhone 15 Pro 和 Pro Max 一样，15 和 15 Plus 是第一批用USB-C取代专有Lightning连接器的iPhone遵守欧盟的指令。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_te

In [7]:
%%time

import faiss

# dimensions of bge-m3
d = 1024
faiss_index = faiss.IndexFlatL2(d)

INFO:faiss.loader:Loading faiss with AVX2 support.
Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.
CPU times: user 17.9 ms, sys: 0 ns, total: 17.9 ms
Wall time: 17.1 ms


运行到下面代码，会一直等待，目前看ollama+bge-m3+llamaindex不可行 -- 不知道问题出在哪个部分

bge-m3的模型文件不是通过ollama加载的，从其他地方下载的。但是curl是可以运行的。

In [10]:
%%time
%%capture
%%bash

curl -s http://192.168.0.72:11435/api/embeddings \
    -H "Content-Type: application/json" \
    -d '{
        "model": "bge-m3",
        "prompt": "Llamas are members of the camelid family"
    }'

CPU times: user 3.85 ms, sys: 0 ns, total: 3.85 ms
Wall time: 33.5 ms


In [ ]:
%%time

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)